In [1]:
import os
import glob
import shutil

file_list = glob.glob('datasets/colo/images/db/*.png')
db = []
ratio = 200
for file in file_list:
    if int(file.split('/')[-1].split('.')[0])%ratio == 0:
        db.append(file)

os.mkdir(os.getcwd()+'/datasets/colo/images/db_'+str(ratio))
for file in db:
    num = file.split('/')[-1].split('.')[0]
    os.symlink(os.getcwd()+'/'+file.split('.')[0]+'.txt', 'datasets/colo/images/db_'+str(ratio)+'/'+num+'.txt')
    os.symlink(os.getcwd()+'/'+file.split('.')[0]+'.tiff', 'datasets/colo/images/db_'+str(ratio)+'/'+num+'.tiff')
    os.symlink(os.getcwd()+'/'+file, 'datasets/colo/images/db_'+str(ratio)+'/'+num+'.png')

FileExistsError: [Errno 17] File exists: '/home/yunho/Hierarchical-Localization/datasets/colo/images/db_200'

In [1]:
from pathlib import Path
from pprint import pformat

from hloc import (
    extract_features,
    match_features,
    localize_guro,
    visualization,
    pairs_from_retrieval,
)

from hloc.visualization import plot_images, read_image, visualize_loc, visualize_loc_3d, visualize_loc_3d_kgnet
import numpy as np
import glob

[2024/07/02 22:12:54 hloc WARNING] hloc requires pycolmap>=0.6.0 but found pycolmap==0.4.0, please upgrade with `pip install --upgrade "pycolmap>=0.6.0"`


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Setup

In [2]:
dataset = Path("datasets/colo/")
images = Path("datasets/colo/images/")
# queries = Path("datasets/guro/query0/")

pairs = Path("pairs/colo/")
loc_pairs = pairs / "pairs-query0-netvlad5.txt"  # top-k retrieved by NetVLAD

outputs = Path("outputs/colo/")  # where everything will be saved
results_linear_5 = outputs / "colo_hloc_superpoint+lightglue_netvlad5_linear.txt"  # the result file
results_linear_5_q0 = outputs / "colo_hloc_superpoint+lightglue_netvlad5_linear_q0.txt"  # the result file
results_linear_5_q1 = outputs / "colo_hloc_superpoint+lightglue_netvlad5_linear_q1.txt"  # the result file
results_linear_5_q2 = outputs / "colo_hloc_superpoint+lightglue_netvlad5_linear_q2.txt"  # the result file
results_linear_5_q3 = outputs / "colo_hloc_superpoint+lightglue_netvlad5_linear_q3.txt"  # the result file
results_cubic_5 = outputs / "colo_hloc_superpoint+lightglue_netvlad5_cubic.txt"  # the result file
results_nn_5 = outputs / "colo_hloc_superpoint+lightglue_netvlad5_nn.txt"  # the result file
results_linear_5_q2_tracking = outputs / "guro_hloc_superpoint+lightglue_kgnet_netvlad.txt"  # the result file

In [4]:
# # list the standard configurations available
# print(f"Configs for feature extractors:\n{pformat(extract_features.confs)}")
# print(f"Configs for feature matchers:\n{pformat(match_features.confs)}")

In [3]:
# images = Path("/home/yunho/guro_db_zed_colmap/images/")

# outputs = Path("/home/yunho/guro_db_zed_colmap/outputs/sfm/")
# sfm_pairs = outputs / "pairs-netvlad.txt"
# sfm_dir = outputs / "sfm_superpoint+lightglue"

# retrieval_conf = extract_features.confs["netvlad"]
retrieval_conf = extract_features.confs["netvlad320"]
# retrieval_conf = extract_features.confs["eigenplaces320"]
feature_conf = extract_features.confs["superpoint_inloc"]
matcher_conf = match_features.confs["superpoint+lightglue"]

# Find image pairs via image retrieval

In [4]:
descriptor_path = extract_features.main(retrieval_conf, images, outputs)

[2024/07/02 21:35:03 hloc INFO] Extracting local features with configuration:
{'model': {'name': 'netvlad'},
 'output': 'global-feats-netvlad320',
 'preprocessing': {'resize_max': 320}}
[2024/07/02 21:35:07 hloc INFO] Found 27103 images in root datasets/colo/images.
[2024/07/02 21:35:10 hloc INFO] Skipping the extraction.


In [11]:
loc_pairs = pairs / "pairs-query1-netvlad320-5.txt"  # top-k retrieved by NetVLAD
pairs_from_retrieval.main(descriptor_path, loc_pairs, num_matched=5, query_prefix="query1", db_prefix="db_200")

[2024/06/03 16:23:44 hloc INFO] Extracting image pairs from a retrieval database.
[2024/06/03 16:23:49 hloc INFO] Found 1545 pairs.


In [ ]:
loc_pairs = pairs / "pairs-query2-salad100.txt"  # top-k retrieved by NetVLAD
# pairs_from_retrieval.main(descriptor_path, loc_pairs, num_matched=100, query_prefix="query2", db_prefix="db")

In [ ]:
loc_pairs = pairs / "pairs-query0-netvlad5-tracking.txt"
pairs_from_retrieval.main_tracking(images, descriptor_path, loc_pairs, num_matched=10, n_neighbor=150, min_matches=200, query_prefix="query0", db_prefix="db")

In [33]:
loc_pairs = pairs / "pairs-query2-netvlad20-tracking.txt"
# pairs_from_retrieval.main_tracking(images, descriptor_path, loc_pairs, num_matched=20, n_neighbor=150, min_matches=450, query_prefix="query2", db_prefix="db")

In [ ]:
loc_pairs = pairs / "pairs-query0-netvlad5-tracking.txt"
pairs_from_retrieval.main_tracking(images, descriptor_path, loc_pairs, num_matched=10, n_neighbor=150, min_matches=150, query_prefix="query2", db_prefix="db")

# Evaluate Retrievals

In [ ]:
# loc_pairs = pairs / "pairs-query2-netvlad20-tracking.txt"
loc_pairs = pairs / "pairs-query2-netvlad20-tracking.txt"

In [18]:
recalls = localize_guro.eval_retrieval_recall(loc_pairs, images, radius=2.5, topks=[1,5,20,30,50,100])
print(str(np.round(recalls, 2))[1:-1].replace(" ", " / "))
recalls = localize_guro.eval_retrieval_recall(loc_pairs, images, radius=5, topks=[1,5,20,30,50,100])
print(str(np.round(recalls, 2))[1:-1].replace(" ", " / "))
recalls = localize_guro.eval_retrieval_recall(loc_pairs, images, radius=10, topks=[1,5,20,30,50,100])
print(str(np.round(recalls, 2))[1:-1].replace(" ", " / "))

0.09 / 0.13 / 0.2 /  / 0.23 / 0.27 / 0.32
0.2 /  / 0.29 / 0.44 / 0.51 / 0.58 / 0.67
0.5 /  / 0.64 / 0.75 / 0.78 / 0.82 / 0.87


In [ ]:
recalls = localize_guro.eval_retrieval_recall(loc_pairs, images, radius=2.5, topks=[1,5,20,30,50,100])
str(np.round(recalls, 2))[1:-1].replace(" ", " / ")

In [ ]:
recalls = localize_guro.eval_retrieval_recall(loc_pairs, images, radius=5, topks=[1,5,20,30,50,100])
str(np.round(recalls, 2))[1:-1].replace(" ", " / ")

In [ ]:
recalls = localize_guro.eval_retrieval_recall(loc_pairs, images, radius=10, topks=[1,5,20,30,50,100])
str(np.round(recalls, 2))[1:-1].replace(" ", " / ")

# Extract and match local features

In [10]:
feature_conf = extract_features.confs["superpoint_inloc512"]
feature_path = extract_features.main(feature_conf, Path("datasets/colo/images/db/"), outputs, depth=True)

[2024/06/03 15:55:24 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 512, 'name': 'superpoint', 'nms_radius': 4},
 'output': 'feats-superpoint-n512-r1600',
 'preprocessing': {'grayscale': True, 'resize_max': 1600}}
[2024/06/03 15:55:24 hloc INFO] Found 12611 images in root datasets/colo/images/db.
[2024/06/03 15:55:28 hloc INFO] Skipping the extraction.


In [7]:
feature_conf = extract_features.confs["superpoint_inloc512"]
feature_path = extract_features.main(feature_conf, images, outputs)

[2024/06/03 16:17:41 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 512, 'name': 'superpoint', 'nms_radius': 4},
 'output': 'feats-superpoint-n512-r1600',
 'preprocessing': {'grayscale': True, 'resize_max': 1600}}
[2024/06/03 16:17:41 hloc INFO] Found 27103 images in root datasets/colo/images.


Loaded SuperPoint model


100%|█████████████████████████████████████████████████| 221/221 [00:03<00:00, 59.43it/s]
[2024/06/03 16:17:49 hloc INFO] Finished exporting features.


In [12]:
match_path = match_features.main(
    matcher_conf, loc_pairs, feature_conf["output"], outputs
)

[2024/06/03 16:23:52 hloc INFO] Matching local features with configuration:
{'model': {'features': 'superpoint', 'name': 'lightglue'},
 'output': 'matches-superpoint-lightglue'}
100%|██████████████████████████████████████████████| 1545/1545 [00:09<00:00, 156.57it/s]
[2024/06/03 16:24:02 hloc INFO] Finished exporting matches.


# Localize

In [ ]:
loc_pairs = pairs / "pairs-query2-netvlad100.txt"

In [ ]:
loc_pairs = pairs / "pairs-query3-netvlad20-tracking150.txt"

In [ ]:
images, feature_path, match_path

In [16]:
loc_pairs

PosixPath('pairs/colo/pairs-query1-netvlad320-100.txt')

In [13]:
localize_guro.main(images, loc_pairs, feature_path, match_path, results_linear_5_q1, topk=5, interp='linear')
pos_err, angle_err, recall = localize_guro.eval_performance(results_linear_5_q1, images)
np.median(pos_err), np.median(angle_err)

[2024/06/03 16:24:03 hloc INFO] Starting localization...
100%|█████████████████████████████████████████████████| 309/309 [00:44<00:00,  6.98it/s]
[2024/06/03 16:24:47 hloc INFO] Writing poses to outputs/colo/colo_hloc_superpoint+lightglue_netvlad5_linear_q1.txt...
[2024/06/03 16:24:47 hloc INFO] Writing logs to outputs/colo/colo_hloc_superpoint+lightglue_netvlad5_linear_q1.txt_logs.pkl...
[2024/06/03 16:24:47 hloc INFO] Done!


Top-5 largest error queries:  ['query1/162.png', 'query1/216.png', 'query1/225.png', 'query1/173.png', 'query1/171.png']
pos mae: 3.665 angle mae: 10.061
pos rmse: 7.965 angle rmse: 35.289


(0.377433260114211, 1.5328980092470172)

In [25]:
localize_guro.main(images, loc_pairs, feature_path, match_path, results_linear_5_q1, topk=20, interp='linear')
pos_err, angle_err, recall = localize_guro.eval_performance(results_linear_5_q1, images)
np.median(pos_err), np.median(angle_err)

[2024/05/23 14:34:04 hloc INFO] Starting localization...
100%|██████████████████████████████████████████████████| 309/309 [02:36<00:00,  1.98it/s]
[2024/05/23 14:36:40 hloc INFO] Writing poses to outputs/colo/colo_hloc_superpoint+lightglue_netvlad5_linear_q1.txt...
[2024/05/23 14:36:40 hloc INFO] Writing logs to outputs/colo/colo_hloc_superpoint+lightglue_netvlad5_linear_q1.txt_logs.pkl...
[2024/05/23 14:36:40 hloc INFO] Done!


Top-5 largest error queries:  ['query1/241.png', 'query1/268.png', 'query1/165.png', 'query1/231.png', 'query1/307.png']
pos mae: 16.685 angle mae: 26.924
pos rmse: 80.508 angle rmse: 55.029


(0.45900962790935923, 2.819959533891892)

In [10]:
localize_guro.main(images, loc_pairs, feature_path, match_path, results_linear_5_q2, topk=20, interp='linear')
pos_err, angle_err, recall = localize_guro.eval_performance(results_linear_5_q2, images)
np.median(pos_err), np.median(angle_err)

[2024/05/08 16:16:50 hloc INFO] Starting localization...
  0%|                                                        | 0/1012 [00:00<?, ?it/s]/home/yunho/Hierarchical-Localization/hloc/localize_guro.py:214: RuntimeWarning: invalid value encountered in matmul
  all_rp3d = np.matmul(all_rp3d_cam, rmat.T) + t_w2c
  0%|                                                | 1/1012 [00:00<12:13,  1.38it/s]

0.7255597114562988


  0%|                                                | 2/1012 [00:01<13:10,  1.28it/s]

0.8229312896728516


  0%|▏                                               | 3/1012 [00:02<12:26,  1.35it/s]

...

100%|█████████████████████████████████████████████| 1012/1012 [11:36<00:00,  1.45it/s]
[2024/05/08 16:28:27 hloc INFO] Writing poses to outputs/colo/colo_hloc_superpoint+lightglue_netvlad5_linear_q2.txt...
[2024/05/08 16:28:27 hloc INFO] Writing logs to outputs/colo/colo_hloc_superpoint+lightglue_netvlad5_linear_q2.txt_logs.pkl...


0.6808662414550781


[2024/05/08 16:28:27 hloc INFO] Done!


Top-5 largest error queries:  ['query2/90.png', 'query2/89.png', 'query2/91.png', 'query2/87.png', 'query2/35.png']
pos mae: 2.245 angle mae: 12.167
pos rmse: 7.007 angle rmse: 37.564


(0.09161113732552975, 2.312241524685481)

In [21]:
localize_guro.main(images, loc_pairs, feature_path, match_path, results_linear_5, topk=20, interp='linear')
pos_err, angle_err, recall = localize_guro.eval_performance(results_linear_5, images)
np.median(pos_err), np.median(angle_err)

[2024/05/09 15:55:57 hloc INFO] Starting localization...
100%|██████████████████████████████████████████████████| 968/968 [08:47<00:00,  1.83it/s]
[2024/05/09 16:04:45 hloc INFO] Writing poses to outputs/colo/colo_hloc_superpoint+lightglue_netvlad5_linear.txt...
[2024/05/09 16:04:45 hloc INFO] Writing logs to outputs/colo/colo_hloc_superpoint+lightglue_netvlad5_linear.txt_logs.pkl...
[2024/05/09 16:04:45 hloc INFO] Done!


Top-5 largest error queries:  ['query0/394.png', 'query0/168.png', 'query0/147.png', 'query0/150.png', 'query0/167.png']
pos mae: 10.065 angle mae: 38.163
pos rmse: 15.582 angle rmse: 64.561


(6.366389754682696, 11.191172449294111)

In [45]:
pos_err, angle_err, recall = localize_guro.eval_performance(results_linear_5_q1, images)
np.median(pos_err), np.median(angle_err), recall
# 0.05, 0.1, 0.25, 0.5, 1m

Top-5 largest error queries:  ['query1/70.png', 'query1/69.png', 'query1/79.png', 'query1/68.png', 'query1/76.png']
pos mae: 0.106 angle mae: 1.062
pos rmse: 0.111 angle rmse: 1.179


(0.1019856725261693,
 0.9181396889086363,
 array([0.00323625, 0.47572816, 1.        , 1.        , 1.        ]))

In [46]:
pos_err, angle_err, recall = localize_guro.eval_performance(results_linear_5_q2, images)
np.median(pos_err), np.median(angle_err), recall

Top-5 largest error queries:  ['query2/90.png', 'query2/89.png', 'query2/91.png', 'query2/87.png', 'query2/35.png']
pos mae: 2.245 angle mae: 12.167
pos rmse: 7.007 angle rmse: 37.564


(0.09161113732552975,
 2.312241524685481,
 array([0.12450593, 0.57509881, 0.86363636, 0.8715415 , 0.87351779]))

In [36]:
pos_err, angle_err, recall = localize_guro.eval_performance(results_linear_5_q2_tracking, images)
np.median(pos_err), np.median(angle_err), recall

Top-5 largest error queries:  ['query2/94.png', 'query2/91.png', 'query2/95.png', 'query2/96.png', 'query2/97.png']
pos mae: 1.867 angle mae: 10.723
pos rmse: 6.805 angle rmse: 36.570


(0.09132738029679965,
 2.199698552590494,
 array([0.13932806, 0.59782609, 0.90217391, 0.91205534, 0.91501976]))

In [ ]:
localize_guro.main(images, loc_pairs, feature_path, match_path, results_linear_5_q3, topk=100, interp='linear')

In [ ]:
localize_guro.main(images, loc_pairs, feature_path, match_path, results_cubic, interp='cubic')

In [ ]:
localize_guro.main(images, loc_pairs, feature_path, match_path, results_nn, interp='nn')

In [ ]:
localize_guro.main_kgnet('hloc/habitat/weights/depth.16.pth', images, loc_pairs, results, interp='linear')

In [ ]:
localize_guro.main_kgnet('hloc/habitat/weights/ckpt.24.pth', images, loc_pairs, results, interp='linear')

# 2D Visualization

In [ ]:
visualize_loc(results_linear, images, n=1, top_k_db=1, seed=2)

In [ ]:
visualize_loc(results_cubic, images, n=1, top_k_db=1, seed=2)

In [ ]:
visualize_loc(results_nn, images, n=1, top_k_db=1, seed=2)

# 3D Visualization
- visualize_loc_3d(result, pcd_dir, images, n, top_k_db, seed, K, ps, size, width, inverval, remove_ceiling)
  - n: # of query image to visualize
  - top_k_db: k of the top-k retrieved images to visualize its camera pose and image matching 
  - seed: random seed to sample queries. If seed=-1, sample from the query trajectory sequentially with interval.
  - K: camera matrix
  - ps: point size when visualizing point clouds
  - size: size of the camera frustum.
  - width: width of the lines in the camera frustum
  - remove_ceiling: boolean to decide whether the ceiling is visualized or not.
  

In [3]:
pcd_dir = dataset / "pointcloud_w_color.pcd"
K = np.array([[526.5249633789062, 0.0,  651.398681640625],
               [0.0, 526.5249633789062, 360.3771057128906],
               [0.0,               0.0,               1.0]])
# K = np.array([[263.27886962890625, 0.0, 325.6990051269531],
#               [0.0, 263.27886962890625, 180.189453125],
#               [0.0, 0.0, 1.0 ]])

In [ ]:
visualize_loc_3d(results, pcd_dir, images, n=300, top_k_db=0, seed=-1, K=K, ps=2, size=5, width=3, interval=10, remove_ceiling=True, ceiling_height=10, downsample=100)

In [ ]:
visualize_loc_3d(results_linear_5_q1, pcd_dir, images, n=300, top_k_db=0, seed=-1, K=K, ps=2, size=5, width=3, interval=10, remove_ceiling=True, ceiling_height=10, downsample=100)

In [ ]:
visualize_loc_3d(results_linear_5_q2, pcd_dir, images, n=300, top_k_db=0, seed=-1, K=K, ps=2, size=5, width=3, interval=10, remove_ceiling=True, ceiling_height=10, downsample=100)

In [ ]:
from hloc.visualization import *
import plotly.graph_objects as go
from PIL import Image

n=300
top_k_db=0
seed=-1
K=K
ps=2
size=5
width=3
interval=10
remove_ceiling=True
ceiling_height=10
downsample=1500
selected=[]
prefix=None
results = results_linear_5_q2_tracking
image_dir = images

pcd = o3d.io.read_point_cloud(str(pcd_dir))
num_points = np.asarray(pcd.points).shape[0]
ind = np.arange(num_points)
np.random.shuffle(ind)
ind = ind[:num_points//downsample]
points = np.asarray(pcd.points)[ind,:]
colors = np.asarray(pcd.colors)[ind,:]
if remove_ceiling:
    colors = colors[points[:,2]<ceiling_height]
    points = points[points[:,2]<ceiling_height]
    
fig = go.FigureWidget()
axes = dict(
    visible=False,
    showbackground=False,
    showgrid=False,
    showline=False,
    showticklabels=True,
    autorange=True,
)
fig.update_layout(
    template="plotly_dark",
    height=800,
    scene_camera=dict(
        eye=dict(x=0.0, y=-0.1, z=-2),
        up=dict(x=0, y=-1.0, z=0),
        projection=dict(type="orthographic"),
    ),
    scene=dict(
        xaxis=axes,
        yaxis=axes,
        zaxis=axes,
        aspectmode="data",
        dragmode="orbit",
    ),
    margin=dict(l=0, r=0, b=0, t=0, pad=0),
    legend=dict(orientation="h", yanchor="top", y=0.99, xanchor="left", x=0.1),
)

img_width = 1280
img_height = 720
scale_factor = 0.0004
fig.add_layout_image(
    dict(
        x=0.02,
        y=img_height * scale_factor * 1.25,
        sizex=img_width * scale_factor,
        sizey=img_height * scale_factor,
#         xref="x domain",
#         yref="y domain",
        opacity=1.0,
        layer="above",
#         sizing="stretch",
        source="https://raw.githubusercontent.com/michaelbabyn/plot_data/master/bridge.jpg")
)
fig.add_layout_image(
    dict(
        x=0.02,
        y=img_height * scale_factor * 2.5,
        sizex=img_width * scale_factor,
        sizey=img_height * scale_factor,
#         xref="x domain",
#         yref="y domain",
        opacity=1.0,
        layer="above",
#         sizing="stretch",
        source="https://raw.githubusercontent.com/michaelbabyn/plot_data/master/bridge.jpg")
)

plot_points(fig, points, colors, ps=ps, name=str(pcd_dir));
fig

In [64]:
with open(str(results) + "_logs.pkl", "rb") as f:
    logs = pickle.load(f)

if not selected:
    queries = list(logs["loc"].keys())
    if prefix:
        queries = [q for q in queries if q.startswith(prefix)]
    if seed == -1:
        q_num = np.array([int(q.split('.')[0].split('/')[1]) for q in queries])
        selected = np.asarray(queries)[q_num.argsort()][0:min(interval*n, len(queries)):interval].tolist()
    else:
        selected = random.Random(seed).sample(queries, min(n, len(queries)))

selected = selected[16:]
        
for qname in selected:
    loc = logs["loc"][qname]
#     fig.layout['images']=[]
    fig.update_layout(
        images=[dict(
                    x=0.02,
                    y=img_height * scale_factor * 2.5,
                    sizex=img_width * scale_factor,
                    sizey=img_height * scale_factor,
                    opacity=1.0,
                    layer="above",
                    source=Image.open(image_dir / qname)),
                dict(
                    x=0.02,
                    y=img_height * scale_factor * 1.25,
                    sizex=img_width * scale_factor,
                    sizey=img_height * scale_factor,
                    opacity=1.0,
                    layer="above",
                    source=Image.open(image_dir / loc['db'][0]))
               ]
    )
 
    visualize_loc_3d_from_log(
        image_dir, qname, loc, fig, K, size, width, top_k_db=0
    );
    


In [ ]:
visualize_loc_3d(results_linear_5, pcd_dir, images, n=158, top_k_db=0, seed=-1, K=K, ps=2, size=5, width=3, interval=10, remove_ceiling=True, ceiling_height=10, downsample=100)

In [ ]:
visualize_loc_3d(results_linear_5_q3, pcd_dir, images, n=1, top_k_db=5, seed=0, prefix='query3/1570.png', K=K, ps=2, size=5, width=3,remove_ceiling=True, ceiling_height=10, downsample=100)

# Quantitative Evaluation - Ransac

In [ ]:
# top-20 query 0

In [ ]:
pos_err, angle_err = localize_guro.eval_performance(results_cubic, images)

In [ ]:
pos_err, angle_err = localize_guro.eval_performance(results_linear, images)

In [ ]:
pos_err, angle_err = localize_guro.eval_performance(results_nn, images)

In [ ]:
# top-5, query 0
pos_err, angle_err = localize_guro.eval_performance(results_linear_5_q0, images)

In [ ]:
# top-5, query 1
pos_err, angle_err = localize_guro.eval_performance(results_linear_5_q1, images)

In [ ]:
# top-5, query 2
pos_err, angle_err = localize_guro.eval_performance(results_linear_5_q2, images)

In [ ]:
# top-5, query 3
pos_err, angle_err = localize_guro.eval_performance(results_linear_5_q3, images)

# Quantitative Evaluation - KG-Net

In [ ]:
pcd_dir = dataset / "pointcloud_w_color.pcd"
K = np.array([[526.5249633789062, 0.0,  651.398681640625],
               [0.0, 526.5249633789062, 360.3771057128906],
               [0.0,               0.0,               1.0]])

In [ ]:
pos_err, angle_err = localize_guro.eval_performance_kgnet(results, images)

In [ ]:
visualize_loc_3d_kgnet(results, pcd_dir, images, n=200, top_k_db=0, seed=-1, K=K, ps=2, size=5, width=3, interval=10, remove_ceiling=True)

In [ ]:
visualize_loc_3d_kgnet(results, pcd_dir, images, n=10, top_k_db=0, seed=-1, K=K, ps=2, size=5, width=3, interval=50, remove_ceiling=True)